hopa1029, klfl1012    

# Übungsblatt 2 - Dispostion
__________________________________________________________________________________________________________________________________________________________________________________________________________________

**DSCB310 – Datenanalyse und Business Intelligence 1**  

Wintersemester 22/23

##### Gliederung des Notebooks:

1. Ziel der Datenanalyse für die Disposition
   
2. Fragestellungen:

   - D1: Prüfen Sie, ob es beim Produkt 13176 auffällige Muster beim Wiederbestellverhalten gibt.
  
   - D2: Spielen die Artikel aus dem Department „produce“ in allen Counties eine gleich große Rolle?
  
   - D3: Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte?
  
3. Zusätzliche Fragestellungen

##### Import des Datensatz: 

**Imports:**

In [4]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px
from plotly.subplots import make_subplots

**Datensatz:**

-- erklärung der einzelnen attribute --

In [ ]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [ ]:
df.reset_index(inplace=True)
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

wir fügen noch ein weiteres Attribut welche die order_size beschreibt hinzu

In [ ]:
df.head()

##  Ziel der Datenanalyse für die Disposition:

#### Aufgabestellung

Erkenntnisse über die regionalen Unterschiede im 
Kaufverhalten der Kunden. Im Fokus sollten dabei die räumlichen Unterschiede des Absatzes bspw. in 
Bezug  auf  die  Produktkategorien,  die  „Gänge“ oder umsatzstarke Produkte stehen.  Die  Disposition 
erhofft sich dadurch eine bessere Versorgung der einzelnen Counties.

Die Disponenten, die für die 
Belieferung  der  Zentren  verantwortlich  sind,  wünschen  sich  Informationen  darüber,  welche  Produkte 
(bzw. Kategorien oder „Gänge“) in welchem County stärker, schwächer oder eher durchschnittlich 
nachgefragt  werden.  Auf  dieser  Basis  soll  die  Belieferung  der  Distributionszentren  optimiert  werden.

Uns sind alle counties in californien gegeben außer butte

### 1. Grundlegende Erkenntnisse

Die stärksten und schwächsten Regionen

### 2. Fragestellungen:

##### **D1:** Prüfen Sie, ob es beim Produkt 13176 auffällige Muster beim Wiederbestellverhalten gibt.

##### **D2:** Spielen die Artikel aus dem Department „produce“ in allen Counties eine gleich große Rolle?

In [ ]:
df_d2 = df.groupby(["county","department"]).agg(anzahl=("department","count")).reset_index()

In [ ]:
counties = df_d2.county.unique().tolist()
for i in counties:
    df_d2.loc[df_d2["county"] == i,"rank"] = df_d2.loc[df_d2["county"] == i].anzahl.rank(method="min",ascending=False)

Annahme: 

Gleiche Rolle bedeutet welchen Platz das department im Vergleich zu den anderen departments in dem county besitzt, basierend auf den Absatzzahlen

In [ ]:
df_d2[df_d2.department == "produce"]

wie man sieht ist das departmen produce, welches aus obst und gemüseprodukten besteht in jedem county außer 4 ausnahmen das Umsatzstärkste

Die Ausnahmen enthalten die counties Del Norte, Sierra, San Bernardino	und Tehama

San Bernardino bestitz wegen seinen niedrigen ausleihzahlen, verglichen zum rest nur wenig statistische signifikanz

##### **D3:** Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte?

In [ ]:
df_d3 = df.groupby(["county","product_name"]).agg(anzahl=("product_name","count")).reset_index()
counties = df_d3.county.unique().tolist()
for i in counties:
    df_d3.loc[df_d3["county"] == i,"rank"] = df_d3.loc[df_d3["county"] == i].anzahl.rank(method="min",ascending=False)

In [ ]:
df_4 = df_d3[df_d3["rank"].isin(range(11))]
df_4 = df_4.sort_values(["county","rank"])

Wir nehmen die Top 10 Produkte je county basierend auf den Absatzzahlen

In [ ]:
df_13 = df_4.groupby("county").agg(coun=("county","count")).reset_index()
df_13[">10"] = df_13.coun > 10
df_4 = pd.merge(df_4,df_13)

In [ ]:
fehler = df_4[df_4[">10"] == True].county.unique()
for i in fehler:
    df_4.drop(df_4[df_4.county == i].tail(df_4[df_4.county == i].coun.values[0]-10).index,inplace=True)

Da es manchmal vorkommt, dass Produkte die gleiche Anzahl an Bestellung innerhalb der Top 10 Produkte eines counties besitzen, lösen wir das Problem indem wir den Dataframe um so viel Zeilen in dem county entfernen, die es mehr als 10 sind. Wir verändern jedoch hierbei nicht den Rang eines Produkts wodurch es totzdem noch zu doppelungen des Rangs innerhalb eines countys vorkommen kann.

In [ ]:
ableitung = []
test2 = []
for i in counties:
    
    for j in counties:
        score = 0
        
        for o in df_4.loc[df_4.county == i,("product_name","rank")].values:
            for z in df_4.loc[df_4.county == j,("product_name","rank")].values:
                if o[0] == z[0]:
                    score = score+1
                    if o[1] == z[1]:
                        score = score+1
        if score < 20:
        
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung.append([score,i,j])

Die ähnlichkeit zwischen counties prüfen wir in dem wir jeden Produkteintrag eines countys mit alledn Produkten des anderesn countys vergleichen.

Falls des Produkt enthalten ist erhöhen wir den unseren score um 1 und wenn das Produkt noch den gleichen Rang besitzt erhöhen wir den Rang ein weiteres mal um 1.

Damit ergibt sich ein Top score von 20(wenn man die gleichen counties vergleichen würde) und eine genauere Ähnlichkeit als wenn man nur auf die verschiedenen Produkte schauen würde und diese beispielsweise mit einer cosinus_similiarity abbilden würde.

In [ ]:
ableitung.sort(reverse=True)

In [ ]:
pd.DataFrame(ableitung,columns=["score","county1","county2"])

### 3. Zusätzliche Fragestellungen:

Welche Produkte, Gänge, Kategorien in welchen counties unter-, über- und nur durchschnittlich vertreten sind

In [ ]:
df_3 = df.groupby(["county","aisle"]).agg(anzahl=("aisle","count")).reset_index()
counties = df_3.county.unique().tolist()
for i in counties:
    df_3.loc[df_3["county"] == i,"rank"] = df_3.loc[df_3["county"] == i].anzahl.rank(method="min",ascending=False)

In [ ]:
df_5 = df_3.groupby("aisle").agg(rank_mean=("rank","mean")).reset_index()
df_6 = pd.merge(df_3,df_5)

In [ ]:
df_8 = df_6.loc[(df_6["rank"] > df_6.rank_mean * 1.5)]
df_9 = df_6.loc[(df_6["rank"] < df_6.rank_mean * 0.5)]

In [ ]:
df_2 = df.copy()
relevant = []
for i in df_9.values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == i[0]))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    if p < 0.05:
        relevant.append(i)

In [ ]:
df_2 = df.copy()
relevant2 = []
for i in df_8.values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county == np.delete(df_test.county.unique(),np.where(df_test.county.unique() == "Alameda"))[-1]) | (df_test.county ==i[0]) ].set_index("county"))
    if p < 0.05:
        relevant2.append(i)

In [ ]:
df_tes1 = pd.DataFrame(relevant,columns=df_9.columns)

In [ ]:
df_tes2 = pd.DataFrame(relevant2,columns=df_9.columns)